#### Installation

In [27]:
%%bash

echo "Installing Python packages..."

# List of Python modules to install
modules=("protobuf>=3.20.2" "mysql-connector-python" "pymysql" "sentence-transformers" "langchain_google_genai" "google-generativeai" "chromadb" "langchain_experimental" "langchain-huggingface")

# Iterate over the modules and install them using pip
for module in "${modules[@]}"; do
  if pip install "$module" --quiet; then
    echo "Successfully installed $module"
  else
    echo "Error installing $module"
    exit 1  # Exit if any module fails to install
  fi
done

echo "All modules installed successfully."

Installing Python packages...
All modules installed successfully.


#### Setup SQL Database

In [28]:
%run ./database_utils.ipynb

MySQL installed and started.
Command executed successfully.
Script executed successfully.
Script executed successfully.
Script executed successfully.
Script executed successfully.
Script executed successfully.
Script executed successfully.
Database and tables set up successfully.

--- Database Setup Report ---
Connected to store_inventory database.
All required tables exist.
Record Counts:
  t_shirts: 180
  discounts: 11
Sample Data:

Table: t_shirts
{'t_shirt_id': 1, 'brand': 'Adidas', 'color': 'Blue', 'size': 'L', 'price': 42, 'stock_quantity': 86}
{'t_shirt_id': 2, 'brand': 'Puma', 'color': 'Navy', 'size': 'M', 'price': 20, 'stock_quantity': 71}
{'t_shirt_id': 3, 'brand': 'Nike', 'color': 'White', 'size': 'XS', 'price': 28, 'stock_quantity': 86}
{'t_shirt_id': 4, 'brand': 'Adidas', 'color': 'Black', 'size': 'S', 'price': 40, 'stock_quantity': 96}
{'t_shirt_id': 5, 'brand': 'Levis', 'color': 'Black', 'size': 'M', 'price': 29, 'stock_quantity': 10}

Table: discounts
{'discount_id': 1,

#### Import Common Modules

In [29]:
import mysql.connector
from pprint import pprint
import warnings
warnings.filterwarnings("ignore")

#### Global Variables

In [30]:
mysql_db_connection = {
    "host": "localhost",
    "user": "root",
    "password": "root",
    "database": "store_inventory"
}

#### Test SQL Database

In [31]:
import pandas as pd

def get_query_result(query):
    connection = mysql.connector.connect(**mysql_db_connection)
    df = pd.read_sql(query, connection)
    return df

In [32]:
sql_query3 ="""SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
        (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levis'
        group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
        """
df = get_query_result(sql_query3)
df.head()

,total_revenue
0,47085.7


#### Playground with Google Generative AI library

In [33]:
from langchain_google_genai import GoogleGenerativeAI
from google.colab import userdata

api_key = userdata.get('GEMINI_KEY')

# Create an instance of the GoogleGenerativeAI class
llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=api_key, temperature=0.2)

In [34]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

database_uri = f"mysql+mysqlconnector://{mysql_db_connection['user']}:{mysql_db_connection['password']}@{mysql_db_connection['host']}/{mysql_db_connection['database']}"

db = SQLDatabase.from_uri(database_uri, sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Peter England','Van Huesen','Levis','Nike','Adidas','Puma') NOT NULL, 
	color ENUM('Red','Blue','Pink','Black','White','Navy') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from t_shirts table:
t

In [35]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [36]:
test_query = db_chain.run("How many t-shirts do we have left for nike in extra small size and white color?, Generate SQL without pre-amble")
print(test_query)



> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for nike in extra small size and white color?, Generate SQL without pre-amble
SQLQuery:SELECT 
    `stock_quantity`
FROM 
    `t_shirts`
WHERE 
    `brand` = 'Nike' AND `color` = 'White' AND `size` = 'XS';
SQLResult: [(86,)]
Answer:86
> Finished chain.
86


#### Few Shot Learning

In [37]:
from decimal import Decimal

def generate_few_shot_list_with_answers(question_query: list[dict[str, str]], db_config: dict) -> list[dict]:
    """
    Generate a few-shot list with answers fetched from a MySQL database.

    :param question_query: List of dictionaries with SQL queries and corresponding questions.
    :param db_config: Dictionary with MySQL database connection parameters.
    :return: List of dictionaries containing questions, queries, and answers.
    """
    # Connect to the MySQL database
    connection = mysql.connector.connect(**db_config)
    cursor = connection.cursor()

    few_shots = []

    for iter, elem in enumerate(question_query, start=1):
        # Initialize answer before the try-except block
        answer = "No result"  # Default value if query fails or returns None
        result = "Result of the SQL query" # Default result value

        # Try to execute the SQL query and fetch the result
        try:
            cursor.execute(elem['SQLQuery'])
            query_res = cursor.fetchone()
            # If query_res is not None, get the first element; otherwise, use the default answer
            if query_res and query_res[0] is not None:
                # Convert Decimal to a clean integer or float
                if isinstance(query_res[0], Decimal):
                    answer = int(query_res[0]) if query_res[0] == query_res[0].to_integral_value() else float(query_res[0])
                else:
                    answer = query_res[0]
        except Exception as e:
            result = f"Error executing query: {e}"

        # Append the result dictionary
        few_shots.append({
            'id': iter,
            'Question': elem['Question'],
            'SQLQuery': elem['SQLQuery'],
            'SQLResult': result,
            'Answer': answer
        })

    # Close the database connection
    cursor.close()
    connection.close()

    return few_shots

In [38]:
question_query = [
    {
        'Question': "How many t-shirts do we have left for Nike in XS size and white color?",
        'SQLQuery': "SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'"
    },
    {
        'Question': "How much is the total price of the inventory for all S-size t-shirts?",
        'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE size = 'S'"
    },
    {
        'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?",
        'SQLQuery': """SELECT SUM(a.total_amount * ((100 - COALESCE(discounts.pct_discount, 0)) / 100)) AS total_revenue
            FROM (
                SELECT SUM(price * stock_quantity) AS total_amount, t_shirt_id
                FROM t_shirts
                WHERE brand = 'Levis'
                GROUP BY t_shirt_id
            ) a
            LEFT JOIN discounts ON a.t_shirt_id = discounts.t_shirt_id
        """
    },
    {
        'Question': "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?",
        'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levis'"
    },
    {
        'Question': "How many white color Levi's shirt I have?",
        'SQLQuery': "SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Levis' AND color = 'White'"
    },
    {
        'Question': "How much sales amount will be generated if we sell all large size t-shirts today in Nike brand after discounts?",
        'SQLQuery': """SELECT SUM(a.total_amount * ((100 - COALESCE(discounts.pct_discount, 0)) / 100)) AS total_revenue
            FROM (
                SELECT SUM(price * stock_quantity) AS total_amount, t_shirt_id
                FROM t_shirts
                WHERE brand = 'Nike' AND size = 'L'
                GROUP BY t_shirt_id
            ) a
            LEFT JOIN discounts ON a.t_shirt_id = discounts.t_shirt_id
        """
    }
]

from pprint import pprint

few_shots = generate_few_shot_list_with_answers(question_query, mysql_db_connection)
pprint(few_shots[:2])

[{'Answer': 86,
  'Question': 'How many t-shirts do we have left for Nike in XS size and white '
              'color?',
  'SQLQuery': "SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Nike' "
              "AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query',
  'id': 1},
 {'Answer': 60631,
  'Question': 'How much is the total price of the inventory for all S-size '
              't-shirts?',
  'SQLQuery': 'SELECT SUM(price * stock_quantity) FROM t_shirts WHERE size = '
              "'S'",
  'SQLResult': 'Result of the SQL query',
  'id': 2}]


In [39]:
from langchain_huggingface import HuggingFaceEmbeddings

# model_name = "sentence-transformers/all-mpnet-base-v2"
model_name = "sentence-transformers/all-MiniLM-L6-v2"
# model_kwargs = {'device': 'cpu'}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    # model_kwargs=model_kwargs,
)

In [40]:
to_vectorize = [' '.join(str(v) for v in elem.values() if v != elem['id']) for elem in few_shots]
print(to_vectorize)

["How many t-shirts do we have left for Nike in XS size and white color? SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' Result of the SQL query 86", "How much is the total price of the inventory for all S-size t-shirts? SELECT SUM(price * stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL query 60631", "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)? SELECT SUM(a.total_amount * ((100 - COALESCE(discounts.pct_discount, 0)) / 100)) AS total_revenue\n            FROM (\n                SELECT SUM(price * stock_quantity) AS total_amount, t_shirt_id\n                FROM t_shirts\n                WHERE brand = 'Levis'\n                GROUP BY t_shirt_id\n            ) a\n            LEFT JOIN discounts ON a.t_shirt_id = discounts.t_shirt_id\n         Result of the SQL query 47085.7", "If we have to sell all the Levi’s T-shirts today. How much 

In [41]:
from langchain.vectorstores import Chroma

fewshot_vectorstore = Chroma.from_texts(to_vectorize, hf, metadatas=[elem for elem in few_shots])

In [42]:
from langchain.prompts import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector(
    vectorstore=fewshot_vectorstore,
    k=2 # pull me 2 similar example
)

example_selector.select_examples({
    'Question': "How many t-shirts do we have left for Nike in XS size and white color?"
})

[{'Answer': 86,
  'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query',
  'id': 1},
 {'Answer': 73,
  'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query',
  'id': 1}]

In [43]:
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [44]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [45]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult", "Answer"],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}"
)

In [46]:
from langchain.prompts import FewShotPromptTemplate

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=_mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"]
)

In [47]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

#### Executor

In [49]:
while True:
    user_input = input("Enter your question (or 'exit' to quit): ")

    if user_input.lower() == 'exit':
        break
    result = new_chain(user_input).get('result', "")
    print(result)

Enter your question (or 'exit' to quit): If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?


> Entering new SQLDatabaseChain chain...
If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SELECT SUM(a.total_amount * ((100 - COALESCE(discounts.pct_discount, 0)) / 100)) AS total_revenue
            FROM (
                SELECT SUM(price * stock_quantity) AS total_amount, t_shirt_id
                FROM t_shirts
                WHERE brand = 'Van Huesen'
                GROUP BY t_shirt_id
            ) a
            LEFT JOIN discounts ON a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('44833.000000'),)]
Answer:44833
> Finished chain.
44833
Enter your question (or 'exit' to quit): How much is the price of all white color levis t shirts?


> Entering new SQLDatabaseChain chain...
How much is the price o